In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# !pip install lasio

# !pip install openpyxl
import lasio


from sklearn.ensemble import RandomForestRegressor
import re
from scipy.interpolate import InterpolatedUnivariateSpline

from scipy.spatial import Voronoi, voronoi_plot_2d

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

from xgboost import XGBClassifier
from joblib import Parallel, delayed
import joblib
import json

In [ ]:

!pip install segyio

!pip install tensorflow
!pip install wellpathpy
!pip install openpyxl


In [ ]:



import segyio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import segyio.segy
import segyio.line as lrr

# from ipywidgets import *
# from ipywidgets import VBox, HBox
# from IPython.display import display, Markdown
# from IPython.html import widgets
# from ipywidgets import FileUpload





import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import logging
from scipy.spatial import distance_matrix

import sklearn.cluster as clust
from sklearn.cluster import KMeans
import time
import math
import warnings
warnings.filterwarnings("ignore")
tf.logging.set_verbosity(tf.logging.ERROR)
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

import matplotlib.pyplot as plt
import wellpathpy as wp
# !pip install lasio
import lasio
from scipy.interpolate import InterpolatedUnivariateSpline
from mpl_toolkits import mplot3d
import re

import xgboost as xgb
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [ ]:
# df=pd.read_excel("../input/voronoi-data/girds.xlsx")

df_hor1 = pd.read_csv(input_location+"H1")
df_hor2 = pd.read_csv(input_location+"H2")
df_hor3 = pd.read_csv(input_location+"H3")
df_hor4 = pd.read_csv(input_location+"H4")



In [ ]:
# E1 = 3273000
# E2 = 3283000

# N1 = 1081000
# N2 = 1091000


# Z1 = 2400
# Z2 = 3000

# xw = 100
# yw = 100
# zw = 600
# E1 = 3273000
# E2 = 3283000
E1 = 3273000
E2 = 3283000
# N1 = 1081000
# N2 = 1091000
N1 = 1082000
N2 = 1092000


Z1 = 2400
Z2 = 3000

xw = 200
yw = 200
zw = 1200

xwidth = (E2-E1)/xw
ywidth = (N2-N1)/yw
zwidth = (Z2-Z1)/zw
print(xwidth)
print(ywidth)
print(zwidth)
ind_point_volume = xwidth*ywidth*zwidth
print(ind_point_volume)



x1 = np.linspace(round(E1), round(E2), xw+1)
y1 = np.linspace(round(N1), round(N2), yw+1)
z1 = np.linspace(round(Z1), round(Z2), zw+1)
X, Y, Z = np.meshgrid(x1, y1, z1)

x=X.flatten()
y=Y.flatten()
z=Z.flatten()
print(len(z))

df_to_pred = pd.DataFrame()

df_to_pred['X'] = x
df_to_pred['Y'] = y
df_to_pred['Z'] = z
df_to_pred.head()

In [ ]:
df_to_pred.shape

In [ ]:
df_to_pred_q  = df_to_pred.copy()
df_to_pred['Hor'] = 0
df_to_pred.head(5)

df_hor1 = df_hor1.drop(['Unnamed: 0'], axis = 1)
df_hor1.head(5)

df_hor2 = df_hor2.drop(['Unnamed: 0'], axis = 1)
df_hor2.head(5)

df_hor3 = df_hor3.drop(['Unnamed: 0'], axis = 1)
df_hor3.head(5)

df_hor4 = df_hor4.drop(['Unnamed: 0'], axis = 1)
df_hor4.head(5)

In [ ]:
#Create a 2d Layer and interpolate

xh = np.linspace(round(E1), round(E2), xw+1)

yh = np.linspace(round(N1), round(N2), yw+1)
print(xh.shape)
# full coordinate arrays

xx, yy = np.meshgrid(xh, yh)
print(xx.shape)

xx=xx.flatten()
yy=yy.flatten()

dfxy = pd.DataFrame()


dfxy['x'] = xx
dfxy['y'] = yy
print(dfxy.shape)
dfxy.head()


model = RandomForestRegressor(random_state=1)
model.fit(df_hor1.iloc[:,:2], df_hor1.iloc[:,2])
# make predictions
predh = model.predict(dfxy)
# evaluate predictions
dfxy['Z1'] = predh
dfxy.head()

model = RandomForestRegressor(random_state=1)
model.fit(df_hor2.iloc[:,:2], df_hor2.iloc[:,2])
# make predictions
predh = model.predict(dfxy.iloc[:,:2])
# evaluate predictions
dfxy['Z2'] = predh
dfxy.head()


model = RandomForestRegressor(random_state=1)
model.fit(df_hor3.iloc[:,:2], df_hor3.iloc[:,2])
# make predictions
predh = model.predict(dfxy.iloc[:,:2])
# evaluate predictions
dfxy['Z3'] = predh
dfxy.head()


model = RandomForestRegressor(random_state=1)
model.fit(df_hor4.iloc[:,:2], df_hor4.iloc[:,2])
# make predictions
predh = model.predict(dfxy.iloc[:,:2])
# evaluate predictions
dfxy['Z4'] = predh
dfxy.head()




In [ ]:
# a = stop

In [ ]:
print(len(dfxy))

new_df_to_pred = pd.DataFrame()

# for i in range(100):
for i in range(0,10000,1):
    
    dfxy_x = dfxy.iloc[i][0]
    dfxy_y = dfxy.iloc[i][1]
    dfxy1 = dfxy.iloc[i][2]
    dfxy2 = dfxy.iloc[i][3]
    dfxy3 = dfxy.iloc[i][4]
    dfxy4 = dfxy.iloc[i][5]
    
    sample_df = df_to_pred[(df_to_pred['X'] == dfxy_x ) & (df_to_pred['Y'] == dfxy_y)]

    sample_df['Z1'] = sample_df['Z']-dfxy1
    sample_df['Z2'] = sample_df['Z']-dfxy2
    sample_df['Z3'] = sample_df['Z']-dfxy3
    sample_df['Z4'] = sample_df['Z']-dfxy4


    new_df_to_pred = new_df_to_pred.append(sample_df)
    print(i)
    

In [ ]:
# new_df_to_pred.to_csv(location+'grid_easy/outputs'+'/f555_1_new1.csv')
# new_df_to_pred = pd.read_csv(location+'grid_easy/outputs'+'/f555_1_new1.csv')

In [ ]:
new_df_to_pred